<a href="https://colab.research.google.com/github/nhat132940/CS114.L21.KHCL/blob/main/Final_Project/Training_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Trên colab: Truy cập Thời gian chạy (Run time)/ Thay đổi loại thời gian chạy( Change run time). Và chọn GPU ở phần Trình tăng tốc phần cứng.

In [ ]:
#Kết nối với Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#Cập nhật các Package
!apt-get update
!apt-get upgrade

In [ ]:
#Tải model về drive
%cd /content/gdrive/MyDrive/
!git clone https://github.com/AlexeyAB/darknet
!rm -rf data #Xóa thư mục data trong thư mục darknet 
!mkdir backup # tạo thư mục backup để lưu các file trọng số sau đào tạo 

In [ ]:
#Tải file yolo.names và yolo.data
%cd /content/gdrive/MyDrive/darknet
!wget https://github.com/nhat132940/CS114.L21.KHCL/tree/main/Final_Project/yolo.names
!wget https://github.com/nhat132940/CS114.L21.KHCL/tree/main/Final_Project/yolo.data

In [ ]:
#Thay thế bằng file Makefile đã chỉnh sửa
%cd /content/gdrive/MyDrive/darknet
!rm Makefile
!wget https://github.com/nhat132940/CS114.L21.KHCL/tree/main/Final_Project/Makefile

In [ ]:
#Thay thế bằng file yolov4-custom.cfg đã chỉnh sửa
%cd /content/gdrive/MyDrive/darknet/cfg
!rm yolov4-custom.cfg
!wget https://github.com/nhat132940/CS114.L21.KHCL/tree/main/Final_Project/yolov4-custom.cfg

In [ ]:
#Tải dataset tại: https://drive.google.com/drive/u/1/folders/1OeK4bTS7ugPlaqtecBGlrJ_ZamtJllIw
#Up file dataset lên thư mục darknet trên drive
#Giải nén file data.zip
%cd /content/gdrive/MyDrive/darknet/
!unzip data.zip

In [ ]:
%cd /content/gdrive/My Drive/darknet/
#Import các thư viện
import glob2
import math  
import os
import numpy as np

#tìm kiếm các file có đuôi dạng: png, jpeg, jpg trong thư mục data
files = []
for ext in ["*.png", "*.jpeg", "*.jpg"]:
  image_files = glob2.glob(os.path.join("data/", ext))
  print(image_files)
  #lưu tất cả các file vào biến files
  files += image_files
  print(len(files))

#Biến nb_val được gán bằng 0.2 độ dài của biến files(chứa tất cả các tên ảnh)
nb_val = math.floor(len(files)*0.2)
#Chọn ngẫu nhiên các ảnh trong biến files
rand_idx = np.random.randint(0, len(files), nb_val)


# Tạo file train.txt
with open("/content/gdrive/My Drive/darknet/train.txt", "w") as f:
  for idx in np.arange(len(files)):
    if (os.path.exists(files[idx][:-3] + "txt")):
      f.write(files[idx]+'\n')


# Tạo file vali.txt
with open("/content/gdrive/My Drive/darknet/val.txt", "w") as f:
  for idx in np.arange(len(files)):
    if (idx in rand_idx) and (os.path.exists(files[idx][:-3] + "txt")):
      f.write(files[idx]+'\n')
  print('done2')

In [ ]:
#Tải pretrain
%cd /content/gdrive/MyDrive/darknet
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
#Chuyển darknet thành folder executable
%cd /content/gdrive/MyDrive/darknet
!make clean
!make
!chmod +x ./darknet

In [ ]:
#Train model
%cd /content/gdrive/MyDrive/darknet
!./darknet detector train yolo.data cfg/yolov4-custom.cfg yolov4.conv.137 -dont_show -map

In [ ]:
# Train tiếp model trong trường hợp bị gián đoạn bằng cách lấy dữ liệu từ file backup 
%cd /content/gdrive/MyDrive/darknet
!./darknet detector train yolo.data cfg/yolov4-custom.cfg backup/yolov4-custom_last.weights -dont_show -map

In [ ]:
#Tham khảo: https://devai.info/2020/12/15/huong-dan-training-object-detection-voi-yolov4-su-dung-google-colab/